In [73]:
import pandas as pd
import numpy as np


df=pd.read_csv("/content/cleaned_churn.csv")



###### FEATURE REVIEW & LEAKAGE CHECK

In [74]:

drop_cols = [
    'customerID',
    'Partner',
    'Dependents',
    'StreamingTV',
    'StreamingMovies',
    'PaperlessBilling',
    'PaymentMethod'
]

df.drop(columns=drop_cols, inplace=True)

In [75]:
categorical_cols = df.select_dtypes(include='object').columns

In [76]:
### Binary encoding   -- reduce dimensionality and btter understaning by model

'''binary_cols = [
    'gender',
    'SeniorCitizen',
    'PhoneService',
    'MultipleLines',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'InternetService',
    'Contract'
]

for col in binary_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})'''


"binary_cols = [\n    'gender',\n    'SeniorCitizen',\n    'PhoneService',\n    'MultipleLines',\n    'OnlineSecurity',\n    'OnlineBackup',\n    'DeviceProtection',\n    'TechSupport',\n    'InternetService',\n    'Contract'\n]\n\nfor col in binary_cols:\n    df[col] = df[col].map({'Yes': 1, 'No': 0})"

In [77]:
print(df['gender'].head())
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})

df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
print(df['gender'].head())


0    Female
1      Male
2      Male
3      Male
4    Female
Name: gender, dtype: object
0    0
1    1
2    1
3    1
4    0
Name: gender, dtype: int64


In [78]:
service_cols = [
    'MultipleLines',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport'
]

for col in service_cols:
    df[col] = df[col].replace({
        'Yes': 1,
        'No': 0,
        'No internet service': 0,
        'No phone service': 0
    })
df.isnull().sum()


/tmp/ipython-input-1930097647.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({


,0
gender,0
SeniorCitizen,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0
OnlineBackup,0
DeviceProtection,0
TechSupport,0


#### One-Hot Encoding (Low Cardinality)

| Feature Type              | Encoding                     |
| ------------------------- | ---------------------------- |
| Binary                    | Binary encoding              |
| Nominal (low cardinality) | One-Hot                      |
| Ordinal                   | Ordinal encoding             |
| High cardinality          | Target / frequency / hashing |


In [79]:
#explicitly mention columns because diff columns need diff encoding and prevent messy

df = pd.get_dummies(
    df,
    columns=['InternetService', 'Contract'],
    drop_first=True
)

In [80]:
#### Identify numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

### Create New Features
df['AvgCharges'] = df['TotalCharges'] / (df['tenure'] + 1)

In [81]:
### Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [82]:
print(df.shape)
df['Churn'].value_counts(normalize=True)

(7043, 17)


,proportion
Churn,
-0.601023,0.73463
1.663829,0.26537


In [85]:
df.select_dtypes(include='object').columns

for col in df.select_dtypes(include='object').columns:
    print(col, df[col].unique())

PhoneService ['No' 'Yes']


In [86]:
df['PhoneService'] = df['PhoneService'].map({'Yes': 1, 'No': 0})
print(df.select_dtypes(include='object').columns)
df['PhoneService'].unique()

Index([], dtype='object')


array([0, 1])

In [88]:
print(df.columns)
df.to_csv('encoded_data.csv', index=False)

Index(['gender', 'SeniorCitizen', 'tenure', 'PhoneService', 'MultipleLines',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'MonthlyCharges', 'TotalCharges', 'Churn',
       'InternetService_Fiber optic', 'InternetService_No',
       'Contract_One year', 'Contract_Two year', 'AvgCharges'],
      dtype='object')
